In [1]:
%tensorflow_version 1.x #use tensorflow magic to use version 1.x in colab

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x #use tensorflow magic to use version 1.x in colab`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [2]:
#imports
import pandas as pd
import numpy as np
import nltk
import io
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.utils import np_utils
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Dropout, Flatten
from keras.models import Sequential, save_model
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [3]:
#traning data
df_train=pd.read_csv("/content/drive/My Drive/minor/english_dataset/english_dataset_prepro.csv")

#drop rows having value NONE in task 3
df_train=df_train[df_train.task_3!="NONE"]
#balancing training data by redundancy
d1=df_train[df_train.task_3=='UNT']
print(len(d1))
d2=df_train[df_train.task_3=='TIN']
print(len(d2))
d1=pd.concat([d1,d1,d1,d1,d1,d1],ignore_index=True)
print(len(d1))
df_train=pd.concat([d1,d2],ignore_index=True)
df_train.head()

220
2041
1320


,text_id,text,task_1,task_2,task_3
0,hasoc_en_135,fck us tho fucktrump,HOF,PRFN,UNT
1,hasoc_en_158,adityarajkaul dukarr 🐷🐽🇵🇰 mullah drying busine...,HOF,PRFN,UNT
2,hasoc_en_250,great thing nazis right wingers alt right type...,HOF,PRFN,UNT
3,hasoc_en_659,fucking idiotic thing threaten proper journali...,HOF,PRFN,UNT
4,hasoc_en_2685,🗣fuck america health care shit motherfuckas ra...,HOF,PRFN,UNT


In [4]:
df_train.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_135,fck us tho fucktrump,HOF,PRFN,UNT
1,hasoc_en_158,adityarajkaul dukarr 🐷🐽🇵🇰 mullah drying busine...,HOF,PRFN,UNT
2,hasoc_en_250,great thing nazis right wingers alt right type...,HOF,PRFN,UNT
3,hasoc_en_659,fucking idiotic thing threaten proper journali...,HOF,PRFN,UNT
4,hasoc_en_2685,🗣fuck america health care shit motherfuckas ra...,HOF,PRFN,UNT


In [5]:
Column_Sequence=np.array(df_train["text"],dtype="str") #storing value  of tweets or text in numpy array of strings

In [6]:
Tokenizer=RegexpTokenizer(r"\w+[a-z]+") #create a reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)

In [7]:
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()                        #lower case
    sen=Tokenizer.tokenize(sen)           #tokenize the sentences.
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)                #append all in one.

In [8]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))                           #fit on training text
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen after padding 300
word_index = tokenizer.word_index      #get a list of all tokens or words in the vocab

In [9]:
#extract word2vec vectors for words in vocab
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors
#load word2vec vectors from file.
word_vectors = KeyedVectors.load_word2vec_format('/content/drive/My Drive/minor/wordtovec/GoogleNews-vectors-negative300.bin.gz', binary=True)

EMBEDDING_DIM=300
vocabulary_size=len(word_index)+1
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector            #store the vector if found.
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)  #store a rndom vector if not found

del(word_vectors)  #del word2vec vectors to free memory

from keras.layers import Embedding
#create an embediing layer from word2vec vectors of our vocab
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
len(tokenizer.word_counts)

9499

In [11]:
X_train=X_t       #training text

In [12]:
X_train.shape

(3361, 300)

In [13]:
Y=df_train["task_3"]    #labels of classes with strings.

In [14]:
Y = Y.map({'UNT':0, 'TIN': 1})   #map the class names to numbers.

In [15]:
# One-hot encode label
Y_train=np_utils.to_categorical(Y)

In [16]:
Y_train.shape

(3361, 2)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.10, random_state=42) #training and validation split of training data

In [18]:
y_train.shape

(3024, 2)

In [19]:
x_train.shape

(3024, 300)

In [20]:
max_len=300 #length of text sequence

In [21]:
x_train

array([[5035, 5036, 5037, ...,    0,    0,    0],
       [ 165,  137,  209, ...,    0,    0,    0],
       [  25,   84,    9, ...,    0,    0,    0],
       ...,
       [ 620, 2105,  119, ...,    0,    0,    0],
       [ 170,   29,   12, ...,    0,    0,    0],
       [ 528,  998, 1291, ...,    0,    0,    0]], dtype=int32)

In [22]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#CNN
sequence_input1 = Input(shape=(max_len,), dtype='int32')  #input layer
embedded_sequences1 = embedding_layer(sequence_input1)
l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences1)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(5)(l_cov3)  # global max pooling
l_flat1 = Flatten()(l_pool3)
l_dense1 = Dense(128, activation='relu')(l_flat1)
preds1 = Dense(2, activation='softmax')(l_dense1)        #output layer

model1 = Model(sequence_input1, preds1)
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Simplified convolutional neural network")
model1.summary()
cp1=ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)  #callback checkpoint


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Simplified convolutional neural network
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          2850000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 296, 128)          192128    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 55, 128)           82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 128)           0         
___

In [23]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, CuDNNGRU, Bidirectional, concatenate
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#BiGRU
sequence_input2 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences2 = embedding_layer(sequence_input2)
gru1=Bidirectional(CuDNNGRU(128, return_sequences=True))(embedded_sequences2)  #gpu optimized gru layer with a bi-directional layer
gru2= Bidirectional(CuDNNGRU(128, return_sequences=True))(gru1)
avg_pool = GlobalAveragePooling1D()(gru2)
max_pool = GlobalMaxPooling1D()(gru2)
conc = concatenate([avg_pool, max_pool])
l_dense2 = Dense(128, activation='relu')(conc)
preds2 = Dense(2, activation='softmax')(l_dense2)  #output layer

model2 = Model(sequence_input2, preds2)
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("GRU")
model2.summary()
cp2=ModelCheckpoint('model_gru.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True) #callback checkpoint

GRU
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     2850000     input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 300, 256)     330240      embedding_1[1][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 300, 256)     296448      bidirectional_1[0][0]            
________________________________________________________________________________________

In [24]:
from keras.layers import Dense, Input, GlobalMaxPooling1D, CuDNNLSTM, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, Bidirectional, concatenate
from keras.models import Model
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
#Bi-LSTM
sequence_input3 = Input(shape=(max_len,), dtype='int32') #input layer
embedded_sequences3 = embedding_layer(sequence_input3)
lstm1 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedded_sequences3) #gpu optimized lstm layer with a bi-directional layer
lstm2= Bidirectional(CuDNNLSTM(128, return_sequences=True))(lstm1)
avg_pool = GlobalAveragePooling1D()(lstm2)
max_pool = GlobalMaxPooling1D()(lstm2)
conc = concatenate([avg_pool, max_pool])
l_dense3 = Dense(128, activation='relu')(conc)
preds3 = Dense(2, activation='softmax')(l_dense3) #output layer

model3 = Model(sequence_input3, preds3)
model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

print("LSTM")
model3.summary()
cp3=ModelCheckpoint('model_lstm.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True) #callback checkpoint

LSTM
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 300)     2850000     input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 300, 256)     440320      embedding_1[2][0]                
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 300, 256)     395264      bidirectional_3[0][0]            
_______________________________________________________________________________________

In [25]:
history1=model1.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=15, batch_size=32,callbacks=[cp1]) #training cnn


Train on 3024 samples, validate on 337 samples
Epoch 1/15
3024/3024 [==============================] - 8s 3ms/step - loss: 0.4639 - acc: 0.7715 - val_loss: 0.1488 - val_acc: 0.9525

Epoch 00001: val_acc improved from -inf to 0.95252, saving model to model_cnn.hdf5
Epoch 2/15
3024/3024 [==============================] - 1s 435us/step - loss: 0.1212 - acc: 0.9587 - val_loss: 0.0502 - val_acc: 0.9881

Epoch 00002: val_acc improved from 0.95252 to 0.98813, saving model to model_cnn.hdf5
Epoch 3/15
3024/3024 [==============================] - 1s 433us/step - loss: 0.0741 - acc: 0.9835 - val_loss: 0.0171 - val_acc: 0.9970

Epoch 00003: val_acc improved from 0.98813 to 0.99703, saving model to model_cnn.hdf5
Epoch 4/15
3024/3024 [==============================] - 1s 440us/step - loss: 0.0516 - acc: 0.9874 - val_loss: 0.3084 - val_acc: 0.9228

Epoch 00004: val_acc did not improve from 0.99703
Epoch 5/15
3024/3024 [==============================] - 1s 435us/step - loss: 0.0589 - acc: 0.9931 - 

In [26]:
history2=model2.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp2]) #training bi-gru

Train on 3024 samples, validate on 337 samples
Epoch 1/7
3024/3024 [==============================] - 23s 8ms/step - loss: 0.4604 - acc: 0.7821 - val_loss: 0.2457 - val_acc: 0.9199
Epoch 2/7


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


3024/3024 [==============================] - 22s 7ms/step - loss: 0.1435 - acc: 0.9547 - val_loss: 0.1195 - val_acc: 0.9644
Epoch 3/7
3024/3024 [==============================] - 22s 7ms/step - loss: 0.0481 - acc: 0.9861 - val_loss: 0.0541 - val_acc: 0.9852
Epoch 4/7
3024/3024 [==============================] - 22s 7ms/step - loss: 0.0309 - acc: 0.9944 - val_loss: 0.0941 - val_acc: 0.9674
Epoch 5/7
3024/3024 [==============================] - 22s 7ms/step - loss: 0.0150 - acc: 0.9977 - val_loss: 0.0382 - val_acc: 0.9881
Epoch 6/7
3024/3024 [==============================] - 22s 7ms/step - loss: 0.0095 - acc: 0.9983 - val_loss: 0.0318 - val_acc: 0.9941
Epoch 7/7
3024/3024 [==============================] - 22s 7ms/step - loss: 0.0103 - acc: 0.9987 - val_loss: 0.0438 - val_acc: 0.9733


In [27]:
history3=model3.fit(x_train, y_train, validation_data=(x_test, y_test),epochs=7, batch_size=32,callbacks=[cp3]) #training bilstm

Train on 3024 samples, validate on 337 samples
Epoch 1/7
3024/3024 [==============================] - 23s 8ms/step - loss: 0.4821 - acc: 0.7599 - val_loss: 0.3369 - val_acc: 0.8546
Epoch 2/7


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


3024/3024 [==============================] - 22s 7ms/step - loss: 0.2296 - acc: 0.9081 - val_loss: 0.1508 - val_acc: 0.9614
Epoch 3/7
3024/3024 [==============================] - 21s 7ms/step - loss: 0.1115 - acc: 0.9597 - val_loss: 0.1007 - val_acc: 0.9733
Epoch 4/7
3024/3024 [==============================] - 21s 7ms/step - loss: 0.0550 - acc: 0.9818 - val_loss: 0.0852 - val_acc: 0.9703
Epoch 5/7
3024/3024 [==============================] - 22s 7ms/step - loss: 0.0405 - acc: 0.9888 - val_loss: 0.0761 - val_acc: 0.9852
Epoch 6/7
3024/3024 [==============================] - 22s 7ms/step - loss: 0.0169 - acc: 0.9950 - val_loss: 0.0947 - val_acc: 0.9703
Epoch 7/7
3024/3024 [==============================] - 21s 7ms/step - loss: 0.0370 - acc: 0.9901 - val_loss: 0.1806 - val_acc: 0.9407


In [28]:
w1=model1.predict(x_test) #predict on val data cnn
w1=np.argmax(w1,axis=1)
w2=model2.predict(x_test) #predict on val data bi-gru
w2=np.argmax(w2,axis=1)
w3=model3.predict(x_test) #predict on val data bi-lstm
w3=np.argmax(w3,axis=1)

In [29]:
from sklearn.metrics import classification_report,confusion_matrix

In [30]:
#true labels for val data
Y_actual=[]
for ix in y_test:
    Y_actual.append(np.argmax(ix))

In [31]:
print("CNN")
print(classification_report(Y_actual,w1))
print("BI-GRU")
print(classification_report(Y_actual,w2))
print("BI-LSTM")
print(classification_report(Y_actual,w3))

CNN
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       149
           1       1.00      0.99      0.99       188

    accuracy                           0.99       337
   macro avg       0.99      0.99      0.99       337
weighted avg       0.99      0.99      0.99       337

BI-GRU
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       149
           1       0.99      0.96      0.98       188

    accuracy                           0.97       337
   macro avg       0.97      0.98      0.97       337
weighted avg       0.97      0.97      0.97       337

BI-LSTM
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       149
           1       1.00      0.89      0.94       188

    accuracy                           0.94       337
   macro avg       0.94      0.95      0.94       337
weighted avg       0.95      0.94      0.94       337



In [32]:
data_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/hasoc2019_en_test-2919_prepro.csv') #testing data
data_test=data_test[data_test.task_3!="NONE"] #droping rows where task_3=NONE

In [33]:
data_test.head()

,text_id,text,task_1,task_2,task_3
4,hasoc_en_568,fuck go back dark ages cow ibnliverealtime rap...,HOF,PRFN,UNT
9,hasoc_en_527,jesus christ christian news illuminati changin...,HOF,HATE,TIN
10,hasoc_en_137,even true muslims supporting zakir knkmow prom...,HOF,PRFN,TIN
11,hasoc_en_606,kerala halal course sickular bastion serve por...,HOF,HATE,TIN
12,hasoc_en_246,look sweetie it’s simple jain ask veg food get...,HOF,PRFN,TIN


In [34]:
data_test.text=data_test.text.astype(str)
df_text=data_test['text']

In [35]:
Column_Sequence=np.array(data_test["text"],dtype="str")
fresh=[]
for ix in Column_Sequence:
    sen=ix.lower()
    sen=Tokenizer.tokenize(sen) #use the reg exp tokenizer object for text to make a list of strings and lowercase text (earlier also removed stopwords but that has been moved to preprocessing code)
    wordss=[(Word) for Word in sen]
    sentance=" ".join(wordss)
    fresh.append(sentance)

In [36]:
tokenizer = keras_text.Tokenizer(char_level = False,lower=True) #create a kerastokenizer object
t=tokenizer.fit_on_texts(list(fresh))
list_tokenized_train = tokenizer.texts_to_sequences(fresh)
X_test = keras_seq.pad_sequences(list_tokenized_train, maxlen=300,padding="post") #maxlen after padding 300

In [37]:
w_test1=model1.predict(X_test)     #predict class of testing data for cnn
w_test1=np.argmax(w_test1,axis=1)
w_test2=model2.predict(X_test)     #predict class of testing data for bi-gru
w_test2=np.argmax(w_test2,axis=1)
w_test3=model3.predict(X_test)     #predict class of testing data for bi-lstm
w_test3=np.argmax(w_test3,axis=1)

In [38]:
data_test['task_3'] = data_test['task_3'].map({'UNT':0, 'TIN': 1}) #map label strings to numbers

In [39]:
y_test_actual=data_test['task_3']

In [40]:
print("CNN")
print(classification_report(y_test_actual,w_test1))
print("BI-GRU")
print(classification_report(y_test_actual,w_test2))
print("BI-LSTM")
print(classification_report(y_test_actual,w_test3))

CNN
              precision    recall  f1-score   support

           0       0.19      0.07      0.10        43
           1       0.85      0.95      0.90       245

    accuracy                           0.82       288
   macro avg       0.52      0.51      0.50       288
weighted avg       0.75      0.82      0.78       288

BI-GRU
              precision    recall  f1-score   support

           0       0.14      0.21      0.17        43
           1       0.85      0.77      0.81       245

    accuracy                           0.69       288
   macro avg       0.49      0.49      0.49       288
weighted avg       0.74      0.69      0.71       288

BI-LSTM
              precision    recall  f1-score   support

           0       0.22      0.35      0.27        43
           1       0.87      0.78      0.82       245

    accuracy                           0.72       288
   macro avg       0.54      0.56      0.55       288
weighted avg       0.77      0.72      0.74       288

